In [ ]:
import sys
import os
notebook_dir = os.path.dirname(os.path.abspath(''))
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

## PyNLO YAG Simulation Setup and Results

In this section I attempt to replicate the experiments in [this](https://opg.optica.org/oe/fulltext.cfm?uri=oe-21-23-28095&id=274338) paper using a PyNLO fiber simulation. The reason for using a fiber simulation instead of a crystal simulation is the fact that PyNLO's crystal sovler only supports 3 wave mixing processes and not 4 wave mixing processes. Since YAG is centro symmetric we only care about the chi 3 processes so we are forced to model the system as a fiber. Since PyNLO solves the 1D nonlinear schrodinger equation throughout the "fiber" we should at the very least get results that qualitatively agree with a more robust 3D crystal solver. I will walk the reader through how I setup this simulation and then show the results.

## Material Considerations (YAG)

### Dispersion Coefficients

PyNLO asks for the disperion coefficients of the YAG plate (or fiber). These values can be calculated from the sellmeier equation.

Sellmeier equation (from Zelmon et al. @ refractive index .com):

$n(\lambda) = [1 + \frac{2.282 \lambda^2}{\lambda^2 - 0.01185} + \frac{3.27644 \lambda^2}{\lambda^2 - 282.734}]^{\frac{1}{2}}$

Propagation constant:

$\beta(\omega) = \frac{n(\lambda(\omega))\omega}{c}$

Dispersion coefficients:

$\beta_{m} = \frac{d^m\beta}{d\omega^m}|_{\omega_{0}}$

For simulation in PyNLO we are interested in $\beta_{2}, \beta_{3}, \beta_{4}$. These values are calculated numerically to be:

$\beta_{2} = -408.15 (ps^2/km)$

$\beta_{3} = 2.54 (ps^3/km)$

$\beta_{4} = -0.02 (ps^4/km)$

The second order dispersion coefficient has good agreement with whats available on refractiveindex.com so its assumed the other two are valid as well.

### Calculating Peak Power for a Guassian Pulse

PyNLO will use a peak power parameter when instantiating a pulse (also accepts an average power parameter). For this analysis a guassian pulse is used. The peak power of a guassian pulse can be calculated with:

$P_{0} = \frac{0.94 E_{pulse}}{\Delta t_{fwhm}}$

This function argument actually does not really matter because once you set the pulse energy is recalculates it automatically.

### Calculating Effective Nonlinear Coefficient

PyNLO will require the effective nonlinearity of the material.

$\gamma = \frac{2 \pi n_{2}}{\lambda A_{eff}}$

Since the objective is to simulate something similar to OPA -> Bulk YAG, I made the effective area not that of a typical fiber, but of the cross sectional area of the beam at the entry face of the material. For example if I am considering a YAG plate at the focus of a beam, I would use the beam waist in the area calculation and assume a circular cross section. If you are doing a tru fiber simulation, and not the crude sim im attempting, you would use the the actual fiber area. Below I have graphed beam radius and gamma as a function of distance from the focus:

<img src="../figures/beam_radius_nonlinearity.png" alt="Beam radius and nonlinearity versus distance from focus" width="1200"/>

$\gamma_{0mm} = 1.548 * 10^{-5}  (1/Wm)$

$\gamma_{0.5mm} = 1.490 * 10^{-5}  (1/Wm)$

$\gamma_{1.0mm} = 1.340 * 10^{-5}  (1/Wm)$

$\gamma_{1.5mm} = 1.147 * 10^{-5}  (1/Wm)$

$\gamma_{2.0mm} = 0.954 * 10^{-5}  (1/Wm)$

$\gamma_{2.5mm} = 0.785 * 10^{-5}  (1/Wm)$

$\gamma_{3.0mm} = 0.645 * 10^{-5}  (1/Wm)$

## Simulation Results

The script that actually runs the simulation can be found in "./PyNLO_YAG_experiment.py". In this section I discuss the results and compare the data from some Lightwave Explorer simulations which attempt to model the same experiment. Overall the PyNLO simulations seem to more closely match the experimental results seen in Hemmer et al. The Lightwave Explorer simulations are limited by a non optimal multiphoton ionization model as well as increased simulation complexity, which would typically be good, but the processes trying to be modeled are highly nonlinear and the simulations quickly become a numerical nightmare. Despite this the PyNLO simulation does have some shortcomings. Since we are *actually* modelling a *fiber* and not a *crystal* the beams longitudal profile is simplied to a cylinder and not a typical guassian. As a result the simulation fails to model any effects which would occur due to the divergence/convergence of the beams radius. The radial profile of the beam is still guassian. PyNLO also fails to model plasma generation and is limited to 1 dimension (1D nonlinear schrodinger equation).

When comparing pulse compression times it is clear that the PyNLO simulation more accurately represent the results from Hemmer et al. While the LWE simulations match qualitatively the times themselves are not accurate at all. Additionaly, the times from PyNLO are in the middle of the conerging/diverging sides of the results from Hemmer et al.

<img src="../figures/pulse_duration_comparison.png" alt="Pulse Duration Comparison" width="1000"/>

When comparing spectrum of self compressed pulses the results from both LWE and PyNLO agree relatively well

<img src="../figures/spectrum_comparison.png" alt="Pulse Duration Comparison" width="1000"/>

.

<img src="../figures/hemmer_spectrum.png" alt="Pulse Duration Comparison" width="600"/>

the intensity profile from PyNLO also look similar to those from Hemmer et al.

<img src="../figures/pynlo_intensity_profiles.png" alt="Pulse Duration Comparison" width="1000"/>

The input pulse parameters, specifically the TOD, and GDD are still being optimized to most closely represent the seed pulse in Hemmer et al.